In [1]:
import os
import cv2
import shutil
from PIL import Image
import numpy as np

In [2]:
def add_salt_pepper_noise(image, salt_prob=0.05, pepper_prob=0.05):
    noisy = np.copy(image)
    total_pixels = image.size

    # Salt noise
    num_salt = np.ceil(salt_prob * total_pixels)

    if len(image.shape) == 3:  # 컬러 이미지
        salt_coords = [np.random.randint(0, i-1, int(num_salt)) for i in image.shape]
        noisy[salt_coords[0], salt_coords[1], :] = 255
    else:  # 흑백 이미지
        salt_coords = [np.random.randint(0, i-1, int(num_salt)) for i in image.shape]
        noisy[salt_coords[0], salt_coords[1]] = 255

    # Pepper noise
    num_pepper = np.ceil(pepper_prob * total_pixels)

    if len(image.shape) == 3:  # 컬러 이미지
        pepper_coords = [np.random.randint(0, i-1, int(num_pepper)) for i in image.shape]
        noisy[pepper_coords[0], pepper_coords[1], :] = 0
    else:  # 흑백 이미지
        pepper_coords = [np.random.randint(0, i-1, int(num_pepper)) for i in image.shape]
        noisy[pepper_coords[0], pepper_coords[1]] = 0

    return noisy

In [3]:
def adjust_brightness(img, factor):
    np_img = np.array(img)
    adjusted = np.clip(np_img + factor, 0, 255)
    
    return adjusted.astype('uint8')  # numpy array를 반환합니다.

In [4]:
def get_average_brightness(img):
    np_img = np.array(img)

    return np.mean(np_img)

In [5]:
def gaussian_blur(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    # 수정가능
    kernel_size = 11
    sigma = 0

    blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)
    
    return blurred_image

In [6]:
def main():
    dir_path   = "../hand_pics"
    file_names = os.listdir(dir_path)

    preprocess_path = "../preprocessed"
    
    # 저장경로 없으면 생성
    if not os.path.exists(preprocess_path):
        os.makedirs(preprocess_path)
    
    # 모든 이미지의 밝기를 200으로 설정하기 위한 상수
    MEAN_BRIGHTNESS = 200
    
    for file_name in file_names:
        image_path    = os.path.join(dir_path, file_name)
        blurred_image = gaussian_blur(image_path)
        
        avg_brightness        = get_average_brightness(blurred_image)
        factor                = MEAN_BRIGHTNESS - avg_brightness
        adjust_brightness_img = adjust_brightness(blurred_image, factor)
        noisy_img             = add_salt_pepper_noise(adjust_brightness_img)
        
        
        save_path = os.path.join(preprocess_path, file_name)

        cv2.imwrite(save_path, noisy_img)

    print("Preprocess Image Done!")

In [7]:
main()

Preprocess Image Done!
